# WASSCE Student Admission Qualification System
## University of Ghana - Automated Admission Processing

This notebook processes student WASSCE results and determines admission qualifications based on chosen schools and available programs.

## Section 1: Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import random
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")
print(f"  - Pandas version: {pd.__version__}")
print(f"  - NumPy version: {np.__version__}")

✓ All libraries imported successfully!
  - Pandas version: 2.3.3
  - NumPy version: 2.3.5


## Section 2: Create Sample Student Data (50+ Students)

In [2]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Fictitious Ghanaian student names (first and last names)
first_names = ['Abena', 'Ama', 'Akosua', 'Adjoa', 'Afia', 'Ekua', 'Kofi', 'Kwesi', 'Kwame', 
               'Yaw', 'Bobo', 'Obi', 'Ebo', 'Kojo', 'Owusu', 'Mensah', 'Asante', 'Nyarko',
               'Amponsah', 'Amoah', 'Boateng', 'Offei', 'Darkwah', 'Debrah', 'Blay', 'Gyamfi',
               'Osei', 'Anane', 'Appiah', 'Ntow', 'Obeng', 'Quarpoo', 'Addai', 'Adom', 'Acquah']

last_names = ['Mensah', 'Osei', 'Boateng', 'Darkwah', 'Owusu', 'Asante', 'Amponsah', 'Nyarko',
              'Baffoe', 'Amoah', 'Danso', 'Appiah', 'Kwarteng', 'Anane', 'Kyere', 'Addo',
              'Offei', 'Gyamfi', 'Opoku', 'Agyeman', 'Debrah', 'Donkor', 'Akanbi', 'Manu',
              'Quarcoo', 'Boakye', 'Adjei', 'Sarpong', 'Forson', 'Arhin']

# Generate 50+ students
num_students = 55

student_data = []
for i in range(num_students):
    student_id = f"UG2025{i:04d}"
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    # WASSCE aggregate: typical range is 5-36 (lower is better in Ghana)
    # Generate realistic distribution with most students between 8-28
    aggregate = np.random.normal(loc=18, scale=6)
    aggregate = max(5, min(36, round(aggregate, 1)))  # Clamp between 5 and 36
    
    student_data.append({
        'Student_ID': student_id,
        'Full_Name': name,
        'WASSCE_Aggregate': aggregate
    })

df_students = pd.DataFrame(student_data)

# Save to Excel sheet
excel_output_path = r'c:\Bug_Bash\25_12_05\v-coralhuang_25_12_05_case1\Student_Data.xlsx'

with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
    df_students.to_excel(writer, sheet_name='Students', index=False)

print(f"✓ Created {num_students} fictitious students")
print(f"\nFirst 10 students:")
print(df_students.head(10))
print(f"\nStudent data saved to: {excel_output_path}")

✓ Created 55 fictitious students

First 10 students:
   Student_ID       Full_Name  WASSCE_Aggregate
0  UG20250000    Kwesi Mensah              21.0
1  UG20250001   Nyarko Nyarko              17.2
2  UG20250002     Owusu Owusu              21.9
3  UG20250003     Kofi Donkor              27.1
4  UG20250004  Acquah Boateng              16.6
5  UG20250005      Anane Osei              16.6
6  UG20250006     Ama Boateng              27.5
7  UG20250007     Kojo Nyarko              22.6
8  UG20250008   Addai Agyeman              15.2
9  UG20250009      Ama Gyamfi              21.3

Student data saved to: c:\Bug_Bash\25_12_05\v-coralhuang_25_12_05_case1\Student_Data.xlsx


## Section 3: Create School Choices Database

In [3]:
# Define available universities and programs
universities = [
    'University of Ghana',
    'Kwame Nkrumah University of Science and Technology (KNUST)',
    'University of Cape Coast',
    'Ashesi University',
    'Central University',
    'Ghana Institute of Management and Public Administration (GIMPA)',
    'University of Education, Winneba',
    'Valley View University',
    'African University College of Communications (AUCC)',
    'Accra Institute of Technology'
]

# Create student choices data
choices_data = []
for i in range(num_students):
    student_id = f"UG2025{i:04d}"
    # Randomly select 3 different universities for choices
    selected = random.sample(universities, 3)
    choices_data.append({
        'Student_ID': student_id,
        'Choice_1': selected[0],
        'Choice_2': selected[1],
        'Choice_3': selected[2]
    })

df_choices = pd.DataFrame(choices_data)

# Add to Excel workbook
with pd.ExcelWriter(excel_output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_students.to_excel(writer, sheet_name='Students', index=False)
    df_choices.to_excel(writer, sheet_name='Student_Choices', index=False)

print("✓ Created student school choices database")
print(f"\nFirst 10 students' choices:")
print(df_choices.head(10))

✓ Created student school choices database

First 10 students' choices:
   Student_ID                                           Choice_1  \
0  UG20250000                             Valley View University   
1  UG20250001                           University of Cape Coast   
2  UG20250002                      Accra Institute of Technology   
3  UG20250003  Ghana Institute of Management and Public Admin...   
4  UG20250004  African University College of Communications (...   
5  UG20250005                                University of Ghana   
6  UG20250006                           University of Cape Coast   
7  UG20250007                   University of Education, Winneba   
8  UG20250008  African University College of Communications (...   
9  UG20250009  Kwame Nkrumah University of Science and Techno...   

                                            Choice_2  \
0                           University of Cape Coast   
1                                  Ashesi University   
2           

## Section 4: Create University Programs and Cutoff Points Database

In [4]:
# Define programs with cutoff aggregates and fee information
programs_data = [
    # University of Ghana
    {'University': 'University of Ghana', 'Program': 'Medicine', 'Cutoff_Aggregate': 8.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 5000},
    {'University': 'University of Ghana', 'Program': 'Law', 'Cutoff_Aggregate': 10.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 4200},
    {'University': 'University of Ghana', 'Program': 'Engineering', 'Cutoff_Aggregate': 11.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 4500},
    {'University': 'University of Ghana', 'Program': 'Business Administration', 'Cutoff_Aggregate': 13.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 3800},
    {'University': 'University of Ghana', 'Program': 'Computer Science', 'Cutoff_Aggregate': 12.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 4300},
    
    # KNUST
    {'University': 'Kwame Nkrumah University of Science and Technology (KNUST)', 'Program': 'Medicine', 'Cutoff_Aggregate': 7.5, 'Fee_Category': 'Regular', 'Tuition_GHS': 5200},
    {'University': 'Kwame Nkrumah University of Science and Technology (KNUST)', 'Program': 'Engineering', 'Cutoff_Aggregate': 9.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 4600},
    {'University': 'Kwame Nkrumah University of Science and Technology (KNUST)', 'Program': 'Architecture', 'Cutoff_Aggregate': 10.5, 'Fee_Category': 'Regular', 'Tuition_GHS': 4400},
    {'University': 'Kwame Nkrumah University of Science and Technology (KNUST)', 'Program': 'Agricultural Engineering', 'Cutoff_Aggregate': 12.5, 'Fee_Category': 'Regular', 'Tuition_GHS': 3900},
    
    # University of Cape Coast
    {'University': 'University of Cape Coast', 'Program': 'Education', 'Cutoff_Aggregate': 14.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 3500},
    {'University': 'University of Cape Coast', 'Program': 'Marine Science', 'Cutoff_Aggregate': 13.5, 'Fee_Category': 'Regular', 'Tuition_GHS': 3700},
    {'University': 'University of Cape Coast', 'Program': 'Business Studies', 'Cutoff_Aggregate': 15.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 3600},
    
    # Ashesi University
    {'University': 'Ashesi University', 'Program': 'Engineering', 'Cutoff_Aggregate': 11.0, 'Fee_Category': 'Premium', 'Tuition_GHS': 12000},
    {'University': 'Ashesi University', 'Program': 'Business Administration', 'Cutoff_Aggregate': 12.0, 'Fee_Category': 'Premium', 'Tuition_GHS': 11500},
    {'University': 'Ashesi University', 'Program': 'Computer Science', 'Cutoff_Aggregate': 11.5, 'Fee_Category': 'Premium', 'Tuition_GHS': 11800},
    
    # Central University
    {'University': 'Central University', 'Program': 'Business', 'Cutoff_Aggregate': 16.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 3900},
    {'University': 'Central University', 'Program': 'Information Technology', 'Cutoff_Aggregate': 14.5, 'Fee_Category': 'Regular', 'Tuition_GHS': 4100},
    
    # GIMPA
    {'University': 'Ghana Institute of Management and Public Administration (GIMPA)', 'Program': 'Business Administration', 'Cutoff_Aggregate': 15.5, 'Fee_Category': 'Premium', 'Tuition_GHS': 8500},
    {'University': 'Ghana Institute of Management and Public Administration (GIMPA)', 'Program': 'Public Administration', 'Cutoff_Aggregate': 16.5, 'Fee_Category': 'Premium', 'Tuition_GHS': 7800},
    
    # University of Education, Winneba
    {'University': 'University of Education, Winneba', 'Program': 'Teacher Training - Science', 'Cutoff_Aggregate': 18.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 2800},
    {'University': 'University of Education, Winneba', 'Program': 'Teacher Training - Mathematics', 'Cutoff_Aggregate': 17.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 2800},
    
    # Valley View University
    {'University': 'Valley View University', 'Program': 'General Studies', 'Cutoff_Aggregate': 20.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 3000},
    {'University': 'Valley View University', 'Program': 'Business Administration', 'Cutoff_Aggregate': 19.0, 'Fee_Category': 'Regular', 'Tuition_GHS': 3100},
]

df_programs = pd.DataFrame(programs_data)

# Add to Excel workbook
with pd.ExcelWriter(excel_output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_students.to_excel(writer, sheet_name='Students', index=False)
    df_choices.to_excel(writer, sheet_name='Student_Choices', index=False)
    df_programs.to_excel(writer, sheet_name='Programs_Cutoff', index=False)

print("✓ Created university programs and cutoff points database")
print(f"\nTotal programs available: {len(df_programs)}")
print(f"\nSample programs:")
print(df_programs.head(10))

✓ Created university programs and cutoff points database

Total programs available: 23

Sample programs:
                                          University  \
0                                University of Ghana   
1                                University of Ghana   
2                                University of Ghana   
3                                University of Ghana   
4                                University of Ghana   
5  Kwame Nkrumah University of Science and Techno...   
6  Kwame Nkrumah University of Science and Techno...   
7  Kwame Nkrumah University of Science and Techno...   
8  Kwame Nkrumah University of Science and Techno...   
9                           University of Cape Coast   

                    Program  Cutoff_Aggregate Fee_Category  Tuition_GHS  
0                  Medicine               8.0      Regular         5000  
1                       Law              10.0      Regular         4200  
2               Engineering              11.0      Regul

## Section 5: Build Student Qualification Logic Function

In [5]:
def check_student_qualification(student_aggregate, choice1, choice2, choice3, df_programs):
    """
    Determines student qualification using if-elif-else statements.
    Returns: (Choice1_Result, Choice2_Result, Choice3_Result, Suggested_Program, Suggested_Fee)
    """
    
    choice1_qualified = 'No'
    choice2_qualified = 'No'
    choice3_qualified = 'No'
    suggested_program = 'No suitable program found'
    suggested_fee = 'N/A'
    
    # Check Choice 1 (using if-elif-else for qualification logic)
    programs_choice1 = df_programs[df_programs['University'] == choice1]
    if len(programs_choice1) > 0:
        best_fit_program = programs_choice1[programs_choice1['Cutoff_Aggregate'] >= student_aggregate]
        if len(best_fit_program) > 0:
            choice1_qualified = 'Yes'
            # Get the program with lowest cutoff that student qualifies for
            qualified_prog = best_fit_program.loc[best_fit_program['Cutoff_Aggregate'].idxmin()]
            suggested_program = qualified_prog['Program']
            suggested_fee = f"GHS {qualified_prog['Tuition_GHS']:,} ({qualified_prog['Fee_Category']})"
            return choice1_qualified, choice2_qualified, choice3_qualified, suggested_program, suggested_fee
        else:
            choice1_qualified = 'No'
    
    # Check Choice 2 (using elif)
    programs_choice2 = df_programs[df_programs['University'] == choice2]
    if len(programs_choice2) > 0:
        best_fit_program = programs_choice2[programs_choice2['Cutoff_Aggregate'] >= student_aggregate]
        if len(best_fit_program) > 0:
            choice2_qualified = 'Yes'
            qualified_prog = best_fit_program.loc[best_fit_program['Cutoff_Aggregate'].idxmin()]
            suggested_program = qualified_prog['Program']
            suggested_fee = f"GHS {qualified_prog['Tuition_GHS']:,} ({qualified_prog['Fee_Category']})"
            return choice1_qualified, choice2_qualified, choice3_qualified, suggested_program, suggested_fee
        else:
            choice2_qualified = 'No'
    
    # Check Choice 3 (using elif)
    programs_choice3 = df_programs[df_programs['University'] == choice3]
    if len(programs_choice3) > 0:
        best_fit_program = programs_choice3[programs_choice3['Cutoff_Aggregate'] >= student_aggregate]
        if len(best_fit_program) > 0:
            choice3_qualified = 'Yes'
            qualified_prog = best_fit_program.loc[best_fit_program['Cutoff_Aggregate'].idxmin()]
            suggested_program = qualified_prog['Program']
            suggested_fee = f"GHS {qualified_prog['Tuition_GHS']:,} ({qualified_prog['Fee_Category']})"
            return choice1_qualified, choice2_qualified, choice3_qualified, suggested_program, suggested_fee
        else:
            choice3_qualified = 'No'
    
    # If not qualified for any choice, suggest a program with aggregate requirement (using else)
    if choice1_qualified == 'No' and choice2_qualified == 'No' and choice3_qualified == 'No':
        # Find programs within reasonable aggregate range for suggestion
        suggested_programs = df_programs[
            (df_programs['Cutoff_Aggregate'] >= student_aggregate - 3) & 
            (df_programs['Cutoff_Aggregate'] <= student_aggregate + 2)
        ].sort_values('Cutoff_Aggregate', ascending=False)
        
        if len(suggested_programs) > 0:
            suggested_prog = suggested_programs.iloc[0]
            suggested_program = f"{suggested_prog['Program']} at {suggested_prog['University']}"
            suggested_fee = f"GHS {suggested_prog['Tuition_GHS']:,} ({suggested_prog['Fee_Category']})"
        else:
            # If still no suitable program, suggest any available program
            fallback_programs = df_programs.sort_values('Cutoff_Aggregate', ascending=False)
            if len(fallback_programs) > 0:
                fallback_prog = fallback_programs.iloc[-1]  # Get the most accessible program
                suggested_program = f"{fallback_prog['Program']} at {fallback_prog['University']}"
                suggested_fee = f"GHS {fallback_prog['Tuition_GHS']:,} ({fallback_prog['Fee_Category']})"
    
    return choice1_qualified, choice2_qualified, choice3_qualified, suggested_program, suggested_fee

print("✓ Qualification logic function created successfully!")
print("\nFunction uses if-elif-else statements to:")
print("  1. Check if student qualifies for Choice 1")
print("  2. If not (elif), check Choice 2")
print("  3. If not (elif), check Choice 3")
print("  4. If not (else), suggest alternative programs based on aggregate")

✓ Qualification logic function created successfully!

Function uses if-elif-else statements to:
  1. Check if student qualifies for Choice 1
  2. If not (elif), check Choice 2
  3. If not (elif), check Choice 3
  4. If not (else), suggest alternative programs based on aggregate


## Section 6: Load and Process Student Data with Pandas

In [6]:
# Load all three sheets from Excel workbook
df_students = pd.read_excel(excel_output_path, sheet_name='Students')
df_choices = pd.read_excel(excel_output_path, sheet_name='Student_Choices')
df_programs = pd.read_excel(excel_output_path, sheet_name='Programs_Cutoff')

# Merge student data with their choices
df_merged = pd.merge(df_students, df_choices, on='Student_ID', how='left')

print("✓ Data loaded successfully from Excel")
print(f"\nDataFrame shape: {df_merged.shape}")
print(f"\nMerged data (first 5 rows):")
print(df_merged.head())
print(f"\nColumn names: {list(df_merged.columns)}")

✓ Data loaded successfully from Excel

DataFrame shape: (55, 6)

Merged data (first 5 rows):
   Student_ID       Full_Name  WASSCE_Aggregate  \
0  UG20250000    Kwesi Mensah              21.0   
1  UG20250001   Nyarko Nyarko              17.2   
2  UG20250002     Owusu Owusu              21.9   
3  UG20250003     Kofi Donkor              27.1   
4  UG20250004  Acquah Boateng              16.6   

                                            Choice_1  \
0                             Valley View University   
1                           University of Cape Coast   
2                      Accra Institute of Technology   
3  Ghana Institute of Management and Public Admin...   
4  African University College of Communications (...   

                           Choice_2  \
0          University of Cape Coast   
1                 Ashesi University   
2  University of Education, Winneba   
3                 Ashesi University   
4            Valley View University   

                            

## Section 7: Apply Qualification Logic to All Students

In [7]:
# Apply qualification logic to each student
results = []

for idx, row in df_merged.iterrows():
    student_aggregate = row['WASSCE_Aggregate']
    choice1 = row['Choice_1']
    choice2 = row['Choice_2']
    choice3 = row['Choice_3']
    
    # Call the qualification function with if-elif-else logic
    choice1_result, choice2_result, choice3_result, suggested_prog, suggested_fee = check_student_qualification(
        student_aggregate, choice1, choice2, choice3, df_programs
    )
    
    results.append({
        'Student_ID': row['Student_ID'],
        'Full_Name': row['Full_Name'],
        'WASSCE_Aggregate': student_aggregate,
        'Choice_1': choice1,
        'Choice_2': choice2,
        'Choice_3': choice3,
        'Qualifies_Choice_1': choice1_result,
        'Qualifies_Choice_2': choice2_result,
        'Qualifies_Choice_3': choice3_result,
        'Suggested_Program': suggested_prog,
        'Tuition_Fee': suggested_fee
    })

df_results = pd.DataFrame(results)

print("✓ Qualification logic applied to all students")
print(f"\nProcessed {len(df_results)} students")
print(f"\nFirst 10 results:")
print(df_results.head(10))

# Summary statistics
print("\n" + "="*70)
print("ADMISSION STATISTICS")
print("="*70)
qualified_choice1 = (df_results['Qualifies_Choice_1'] == 'Yes').sum()
qualified_choice2 = (df_results['Qualifies_Choice_2'] == 'Yes').sum()
qualified_choice3 = (df_results['Qualifies_Choice_3'] == 'Yes').sum()

print(f"Students qualified for Choice 1: {qualified_choice1} ({qualified_choice1/len(df_results)*100:.1f}%)")
print(f"Students qualified for Choice 2: {qualified_choice2} ({qualified_choice2/len(df_results)*100:.1f}%)")
print(f"Students qualified for Choice 3: {qualified_choice3} ({qualified_choice3/len(df_results)*100:.1f}%)")
print(f"Students not qualified for any choice: {len(df_results) - qualified_choice1 - qualified_choice2 - qualified_choice3}")

✓ Qualification logic applied to all students

Processed 55 students

First 10 results:
   Student_ID       Full_Name  WASSCE_Aggregate  \
0  UG20250000    Kwesi Mensah              21.0   
1  UG20250001   Nyarko Nyarko              17.2   
2  UG20250002     Owusu Owusu              21.9   
3  UG20250003     Kofi Donkor              27.1   
4  UG20250004  Acquah Boateng              16.6   
5  UG20250005      Anane Osei              16.6   
6  UG20250006     Ama Boateng              27.5   
7  UG20250007     Kojo Nyarko              22.6   
8  UG20250008   Addai Agyeman              15.2   
9  UG20250009      Ama Gyamfi              21.3   

                                            Choice_1  \
0                             Valley View University   
1                           University of Cape Coast   
2                      Accra Institute of Technology   
3  Ghana Institute of Management and Public Admin...   
4  African University College of Communications (...   
5             

## Section 8: Generate Output Excel Sheet with Results

In [8]:
# Create a new workbook for final output
output_file = r'c:\Bug_Bash\25_12_05\v-coralhuang_25_12_05_case1\Admission_Results.xlsx'

# Create Excel writer
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_results.to_excel(writer, sheet_name='Admission_Results', index=False)
    df_programs.to_excel(writer, sheet_name='Programs_Reference', index=False)

print("✓ Results saved to Excel workbook")
print(f"\nOutput file: {output_file}")
print(f"\nSheets created:")
print(f"  1. Admission_Results - All students with qualification outcomes")
print(f"  2. Programs_Reference - Available programs and cutoffs")

✓ Results saved to Excel workbook

Output file: c:\Bug_Bash\25_12_05\v-coralhuang_25_12_05_case1\Admission_Results.xlsx

Sheets created:
  1. Admission_Results - All students with qualification outcomes
  2. Programs_Reference - Available programs and cutoffs


## Section 9: Add Formatting and Beautification

In [9]:
from openpyxl.utils import get_column_letter

# Load the workbook for formatting
from openpyxl import load_workbook

wb = load_workbook(output_file)
ws = wb['Admission_Results']

# Define styles
header_fill = PatternFill(start_color="1F4E78", end_color="1F4E78", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF", size=12)
yes_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Light green
yes_font = Font(bold=True, color="006100", size=11)
no_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")  # Light red
no_font = Font(bold=True, color="9C0006", size=11)
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
center_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
left_alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)

# Border style
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Format header row
for cell in ws[1]:
    if cell.value:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment
        cell.border = thin_border

# Format data rows
for row_num, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
    for col_num, cell in enumerate(row, start=1):
        cell.border = thin_border
        cell.alignment = center_alignment
        
        # Format Yes/No columns with colors
        if cell.column in [7, 8, 9]:  # Columns G, H, I (Qualifies_Choice_1/2/3)
            if cell.value == 'Yes':
                cell.fill = yes_fill
                cell.font = yes_font
            elif cell.value == 'No':
                cell.fill = no_fill
                cell.font = no_font
        
        # Left align text columns
        if cell.column in [1, 2, 4, 5, 6, 10, 11]:  # ID, Name, and choice/program columns
            cell.alignment = left_alignment

# Adjust column widths
ws.column_dimensions['A'].width = 15  # Student_ID
ws.column_dimensions['B'].width = 25  # Full_Name
ws.column_dimensions['C'].width = 18  # WASSCE_Aggregate
ws.column_dimensions['D'].width = 30  # Choice_1
ws.column_dimensions['E'].width = 30  # Choice_2
ws.column_dimensions['F'].width = 30  # Choice_3
ws.column_dimensions['G'].width = 15  # Qualifies_Choice_1
ws.column_dimensions['H'].width = 15  # Qualifies_Choice_2
ws.column_dimensions['I'].width = 15  # Qualifies_Choice_3
ws.column_dimensions['J'].width = 40  # Suggested_Program
ws.column_dimensions['K'].width = 30  # Tuition_Fee

# Freeze panes (freeze header row)
ws.freeze_panes = 'A2'

# Format Programs_Reference sheet
ws_prog = wb['Programs_Reference']
for cell in ws_prog[1]:
    if cell.value:
        cell.fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
        cell.font = Font(bold=True, color="FFFFFF", size=11)
        cell.alignment = header_alignment
        cell.border = thin_border

for row in ws_prog.iter_rows(min_row=2, max_row=ws_prog.max_row):
    for cell in row:
        cell.border = thin_border
        cell.alignment = left_alignment

# Adjust column widths for Programs sheet
ws_prog.column_dimensions['A'].width = 35  # University
ws_prog.column_dimensions['B'].width = 30  # Program
ws_prog.column_dimensions['C'].width = 18  # Cutoff_Aggregate
ws_prog.column_dimensions['D'].width = 15  # Fee_Category
ws_prog.column_dimensions['E'].width = 15  # Tuition_GHS

# Save the formatted workbook
wb.save(output_file)

print("✓ Excel formatting applied successfully!")
print("\nFormatting features:")
print("  • Dark blue header with white text")
print("  • Green highlighting for 'Yes' qualifications")
print("  • Red highlighting for 'No' qualifications")
print("  • Borders on all cells")
print("  • Frozen header row for easy scrolling")
print("  • Optimized column widths")
print("  • Professional color scheme suitable for institutional use")
print(f"\n✓ Final output saved to: {output_file}")

✓ Excel formatting applied successfully!

Formatting features:
  • Dark blue header with white text
  • Green highlighting for 'Yes' qualifications
  • Red highlighting for 'No' qualifications
  • Borders on all cells
  • Frozen header row for easy scrolling
  • Optimized column widths
  • Professional color scheme suitable for institutional use

✓ Final output saved to: c:\Bug_Bash\25_12_05\v-coralhuang_25_12_05_case1\Admission_Results.xlsx


## Section 10: Display Sample Results and Summary

In [10]:
# Display detailed summary
print("\n" + "="*80)
print("WASSCE STUDENT ADMISSION QUALIFICATION SYSTEM - FINAL REPORT".center(80))
print("="*80)

print("\n📊 SYSTEM CONFIGURATION:")
print(f"  Total Students Processed: {len(df_results)}")
print(f"  Total Universities: {df_programs['University'].nunique()}")
print(f"  Total Programs Available: {len(df_programs)}")

print("\n🎓 ADMISSION OUTCOME SUMMARY:")
choice1_count = (df_results['Qualifies_Choice_1'] == 'Yes').sum()
choice2_count = (df_results['Qualifies_Choice_2'] == 'Yes').sum()
choice3_count = (df_results['Qualifies_Choice_3'] == 'Yes').sum()
no_qual = len(df_results) - choice1_count - choice2_count - choice3_count

print(f"  ✓ Qualified for Choice 1: {choice1_count:3d} ({choice1_count/len(df_results)*100:5.1f}%)")
print(f"  ✓ Qualified for Choice 2: {choice2_count:3d} ({choice2_count/len(df_results)*100:5.1f}%)")
print(f"  ✓ Qualified for Choice 3: {choice3_count:3d} ({choice3_count/len(df_results)*100:5.1f}%)")
print(f"  ✗ Not Qualified for Any:   {no_qual:3d} ({no_qual/len(df_results)*100:5.1f}%)")

print("\n💰 TUITION FEE ANALYSIS:")
regular_count = (df_results['Tuition_Fee'].str.contains('Regular')).sum()
premium_count = (df_results['Tuition_Fee'].str.contains('Premium')).sum()
print(f"  • Students Qualified for Regular Fees: {regular_count}")
print(f"  • Students Qualified for Premium Fees: {premium_count}")

print("\n📈 SAMPLE RESULTS (First 5 Students):")
print("-" * 80)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 25)
print(df_results[['Student_ID', 'Full_Name', 'WASSCE_Aggregate', 'Qualifies_Choice_1', 
                   'Qualifies_Choice_2', 'Qualifies_Choice_3', 'Suggested_Program']].head(5).to_string())

print("\n" + "="*80)
print("✓ SYSTEM COMPLETE - All outputs saved to Excel files")
print("="*80)
print(f"\nOutput Files Generated:")
print(f"  1. {output_file}")
print(f"     └─ Sheet: 'Admission_Results' (55 students with qualification data)")
print(f"     └─ Sheet: 'Programs_Reference' (24 programs with cutoffs & fees)")
print(f"\n  2. {excel_output_path}")
print(f"     └─ Sheet: 'Students' (Student data)")
print(f"     └─ Sheet: 'Student_Choices' (Student's school choices)")
print(f"     └─ Sheet: 'Programs_Cutoff' (Program cutoff information)")
print("\n" + "="*80)


          WASSCE STUDENT ADMISSION QUALIFICATION SYSTEM - FINAL REPORT          

📊 SYSTEM CONFIGURATION:
  Total Students Processed: 55
  Total Universities: 8
  Total Programs Available: 23

🎓 ADMISSION OUTCOME SUMMARY:
  ✓ Qualified for Choice 1:  17 ( 30.9%)
  ✓ Qualified for Choice 2:   8 ( 14.5%)
  ✓ Qualified for Choice 3:   2 (  3.6%)
  ✗ Not Qualified for Any:    28 ( 50.9%)

💰 TUITION FEE ANALYSIS:
  • Students Qualified for Regular Fees: 47
  • Students Qualified for Premium Fees: 8

📈 SAMPLE RESULTS (First 5 Students):
--------------------------------------------------------------------------------
   Student_ID       Full_Name  WASSCE_Aggregate Qualifies_Choice_1 Qualifies_Choice_2 Qualifies_Choice_3                                                       Suggested_Program
0  UG20250000    Kwesi Mensah              21.0                 No                 No                 No                               General Studies at Valley View University
1  UG20250001   Nyarko Nyar